In [1]:
import sys
from spacy import displacy
import ipywidgets as widgets
from IPython.display import display, HTML
import json

sys.path.append("../")
from src.extraction.extraction import Extraction
from src.config.experimental_config import load_experimental_config
from src.config.global_config import load_global_config

global_config_path = "../config/global_config.yaml"
global_config = load_global_config(global_config_path)

default_config_path = "../config/experimental_config.yaml"
experimental_config = load_experimental_config(default_config_path)

In [2]:
data_path = "../example_output/example_pipeline_14_05_24/llm.json"

with open(data_path) as file:
    data = json.load(file)

In [3]:
experimental_config.extraction.entity_list = ["nhs number"]

In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
from spacy import displacy

# Extract results
results = Extraction(
    global_config=global_config,
    extractionconfig=experimental_config.extraction,
    llm_input=data,
).run_or_load(save=False)


def initialise(index):
    global entity_dicts
    global current_index
    entity_dicts = []
    current_index = index

    for i in range(len(results)):
        entity_dicts.append(
            {
                "text": data[i],
                "ents": results[i]["Entities"],
            }
        )

    update_entity_display()


# Define a function to update and display the displaCy visualization
def update_visualization(dic):
    with displacy_output:
        displacy_output.clear_output()
        try:
            # Render the visualization directly to the output area
            displacy.render(dic, manual=True, style="ent", jupyter=True)
        except Exception as e:
            print("Error rendering visualization:", e)


# Define a function to handle button click for changing to next review
def on_next_button_click(b):
    global current_index
    current_index = (current_index + 1) % len(entity_dicts)
    dic = entity_dicts[current_index]
    update_visualization(dic)


# Define a function to handle button click for changing to previous review
def on_previous_button_click(b):
    global current_index
    current_index = (current_index - 1) % len(entity_dicts)
    dic = entity_dicts[current_index]
    update_visualization(dic)


# Define a function to handle button click for submitting note
def on_submit_button_click(b):
    note_text = note_input.value.strip()
    if note_text:
        experimental_config.extraction.entity_list.append(note_text)
    update_entity_display()


def on_remove_button_click(b):
    note_text = note_input.value.strip()
    if note_text:
        try:
            experimental_config.extraction.entity_list.remove(note_text)
            update_entity_display()
        except:
            pass


def update_entity_display():
    with entities_output:
        entities_output.clear_output()
        display(
            HTML(
                f"Current Entities: {', '.join(experimental_config.extraction.entity_list)}"
            )
        )


def NER(b):

    with displacy_output:
        displacy_output.clear_output()
        display(HTML("Extracting Entities..."))

    global results
    results = Extraction(
        global_config=global_config,
        extractionconfig=experimental_config.extraction,
        llm_input=data,
    ).run_or_load(save=False)

    initialise(current_index)
    update_visualization(entity_dicts[current_index])


# Define initial widgets
next_button = widgets.Button(description="Next Note")
previous_button = widgets.Button(description="Previous Note")
NER_button = widgets.Button(description="NER", button_style="success")
displacy_output = widgets.Output()
note_input = widgets.Text(placeholder="New Entity type")
submit_button = widgets.Button(
    description="Add Entity", button_style="primary"
)
remove_button = widgets.Button(
    description="Remove Entity", button_style="danger"
)
entities_output = widgets.Output()

initialise(0)

# Set the style for the output widget
displacy_output.layout = widgets.Layout(
    height="300px", overflow_y="scroll", overflow_x="auto"
)

# Attach the button click event handler
next_button.on_click(on_next_button_click)
previous_button.on_click(on_previous_button_click)
submit_button.on_click(on_submit_button_click)
remove_button.on_click(on_remove_button_click)
NER_button.on_click(NER)

input_box = widgets.HBox([note_input, submit_button, remove_button])
navigation_box = widgets.HBox([previous_button, next_button, NER_button])
# Display the initial button and output area

display(displacy_output)
display(navigation_box)
display(input_box)
display(entities_output)

# Initialize the visualization with the first piece of data
update_visualization(entity_dicts[current_index])

/mnt/c/Users/Will Poulett/Documents/PrivFingerprint/privfp-experiments/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/mnt/c/Users/Will Poulett/Documents/PrivFingerprint/privfp-experiments/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Output(layout=Layout(height='300px'))

Output()